## *FBE551: Quantitative Investing* Final Project

**Team Name:**

    JPMarshall

**Team Members:**

    Yi LIU
    Hao WANG
    Siqin YANG
    Sijia ZENG


In this notebook, we implement four individual strategies, **Reversal, Payout Yield, Volatility, Momentum** separately and use results for further combination.

In [1]:
import numpy as np
import pandas as pd

## Data Preparation

In [2]:
data = pd.read_feather('./Data/crspm.feather')
data = data.loc[data['DATE']>'1993-01-27']

### Data Interpretation

In [3]:
data

,PERMNO,DATE,PRC,SHROUT,SHRCD,EXCHCD,SICCD,VOL,RET,DIVAMT
104,10001.0,1993-01-29,-14.000000,1080.0,11.0,3.0,4920.0,44.0,0.000000,NaN
105,10001.0,1993-02-26,14.250000,1080.0,11.0,3.0,4920.0,23.0,0.017857,NaN
106,10001.0,1993-03-31,14.250000,1075.0,11.0,3.0,4920.0,137.0,0.011053,0.1575
107,10001.0,1993-04-30,-15.250000,1075.0,11.0,3.0,4920.0,473.0,0.070175,NaN
108,10001.0,1993-05-28,-15.125000,1075.0,11.0,3.0,4920.0,126.0,-0.008197,NaN
...,...,...,...,...,...,...,...,...,...,...
4395316,93436.0,2020-08-31,498.320007,931809.0,11.0,3.0,9999.0,4051970.0,0.741452,0.0000
4395317,93436.0,2020-09-30,429.010010,948000.0,11.0,3.0,9999.0,17331954.0,-0.139087,NaN
4395318,93436.0,2020-10-30,388.040009,947901.0,11.0,3.0,9999.0,8330610.0,-0.095499,NaN
4395319,93436.0,2020-11-30,567.599976,947901.0,11.0,3.0,9999.0,7811501.0,0.462736,NaN


| Column Name | Description |
| :----- | :---- |
| permno | a unique security identifier |
| date | the date |
| prc | closing price; if negative, then the midpoint of the closing bid-ask spread; if you use this, take the absolute value |
| shrout | number of shares outstanding |
|...|...|
| vol | trading volume in shares |
| ret | rate of return |
| divamt | amount of any dividend paid |  

### Data Preprocess

duplicates exist for same PERMNO & DATE

In [4]:
data.duplicated(['PERMNO', 'DATE']).sum()

24315

among duplicates, choose the one with highest MV

In [5]:
# Define MV
data['MV']  = data['SHROUT'] * data['PRC']
data = data.sort_values(by=['PERMNO', 'DATE', 'MV'])
data = data.groupby(['PERMNO', 'DATE']).tail(1)
data.duplicated(['PERMNO', 'DATE']).sum()

0

## Individual Strategies

### Reversal

In [6]:
df = data[['PERMNO','DATE','PRC','VOL','RET','SHROUT']].copy()

In [7]:
df.isna().sum()

PERMNO        0
DATE          0
PRC       68052
VOL       52827
RET       85893
SHROUT    18610
dtype: int64

drop the missing returns

In [8]:
df.dropna(subset=['RET'], inplace=True)
df.isna().sum()

PERMNO    0
DATE      0
PRC       0
VOL       0
RET       0
SHROUT    0
dtype: int64

re_index the dataframe based on each ETF as a group 

In [9]:
df = df.set_index(['PERMNO','DATE'])
df

PRC         VOL       RET    SHROUT
PERMNO  DATE                                                  
10001.0 1993-01-29  -14.000000        44.0  0.000000    1080.0
        1993-02-26   14.250000        23.0  0.017857    1080.0
        1993-03-31   14.250000       137.0  0.011053    1075.0
        1993-04-30  -15.250000       473.0  0.070175    1075.0
        1993-05-28  -15.125000       126.0 -0.008197    1075.0
...                        ...         ...       ...       ...
93436.0 2020-08-31  498.320007   4051970.0  0.741452  931809.0
        2020-09-30  429.010010  17331954.0 -0.139087  948000.0
        2020-10-30  388.040009   8330610.0 -0.095499  947901.0
        2020-11-30  567.599976   7811501.0  0.462736  947901.0
        2020-12-31  705.669983  11962716.0  0.243252  947901.0

[2471978 rows x 4 columns]

compute the moving average return of past $N$ days

In [10]:
N_range = range(1,11)
for N in N_range:
    df[f'ret_ma{N}'] = df.groupby('PERMNO')['RET'].apply(lambda x: x.rolling(N).mean())
    df[f'ret_ma{N}_lag'] = df[f'ret_ma{N}'].groupby('PERMNO').shift()
    print(f'Moving Average window size {N} done.')

Moving Average window size 1 done.
Moving Average window size 2 done.
Moving Average window size 3 done.
Moving Average window size 4 done.
Moving Average window size 5 done.
Moving Average window size 6 done.
Moving Average window size 7 done.
Moving Average window size 8 done.
Moving Average window size 9 done.
Moving Average window size 10 done.


In [11]:
df.head()

PRC    VOL       RET  SHROUT   ret_ma1  ret_ma1_lag  \
PERMNO  DATE                                                                 
10001.0 1993-01-29 -14.000   44.0  0.000000  1080.0  0.000000          NaN   
        1993-02-26  14.250   23.0  0.017857  1080.0  0.017857     0.000000   
        1993-03-31  14.250  137.0  0.011053  1075.0  0.011053     0.017857   
        1993-04-30 -15.250  473.0  0.070175  1075.0  0.070175     0.011053   
        1993-05-28 -15.125  126.0 -0.008197  1075.0 -0.008197     0.070175   

                     ret_ma2  ret_ma2_lag   ret_ma3  ret_ma3_lag  ...  \
PERMNO  DATE                                                      ...   
10001.0 1993-01-29       NaN          NaN       NaN          NaN  ...   
        1993-02-26  0.008929          NaN       NaN          NaN  ...   
        1993-03-31  0.014455     0.008929  0.009637          NaN  ...   
        1993-04-30  0.040614     0.014455  0.033028     0.009637  ...   
        1993-05-28  0.030989     0.040614  0.024344     0.033028  ...   

                    ret_ma6  ret_ma6_lag  ret_ma7  ret_ma7_lag  ret_ma8  \
PERMNO  DATE                                                              
10001.0 1993-01-29      NaN          NaN      NaN          NaN      NaN   
        1993-02-26      NaN          NaN      NaN          NaN      NaN   
        1993-03-31      NaN          NaN      NaN          NaN      NaN   
        1993-04-30      NaN          NaN      NaN          NaN      NaN   
        1993-05-28      NaN          NaN      NaN          NaN      NaN   

                    ret_ma8_lag  ret_ma9  ret_ma9_lag  ret_ma10  ret_ma10_lag  
PERMNO  DATE                                                                   
10001.0 1993-01-29          NaN      NaN          NaN       NaN           NaN  
        1993-02-26          NaN      NaN          NaN       NaN           NaN  
        1993-03-31          NaN      NaN          NaN       NaN           NaN  
        1993-04-30          NaN      NaN          NaN       NaN           NaN  
        1993-05-28          NaN      NaN          NaN       NaN           NaN  

[5 rows x 24 columns]

check NAs resulted from df.shift()

In [12]:
df.isna().sum()

PRC                  0
VOL                  0
RET                  0
SHROUT               0
ret_ma1              0
ret_ma1_lag      24089
ret_ma2          24089
ret_ma2_lag      48040
ret_ma3          48040
ret_ma3_lag      71822
ret_ma4          71822
ret_ma4_lag      95474
ret_ma5          95474
ret_ma5_lag     119028
ret_ma6         119028
ret_ma6_lag     142454
ret_ma7         142454
ret_ma7_lag     165735
ret_ma8         165735
ret_ma8_lag     188890
ret_ma9         188890
ret_ma9_lag     211898
ret_ma10        211898
ret_ma10_lag    234778
dtype: int64

play a more conversative way: 

in order to get a more complete list of ranks, drop the missing values when calculating quantiles based on different past *N* days rather than the biggest *N* day right now

reorder index levels, otherwise *ValueError: cannot handle a non-unique multi-index!*

In [13]:
df = df.reorder_levels(['DATE', 'PERMNO'])
df.sort_index(inplace=True)

In [14]:
df

PRC         VOL       RET    SHROUT   ret_ma1  \
DATE       PERMNO                                                          
1993-01-29 10001.0  -14.000000        44.0  0.000000    1080.0  0.000000   
           10002.0   -9.500000        19.0  0.055556    1442.0  0.055556   
           10003.0    4.750000      1944.0  0.225806    2194.0  0.225806   
           10009.0   13.000000       495.0  0.061224    1158.0  0.061224   
           10010.0   12.000000     23860.0  0.010526    7948.0  0.010526   
...                        ...         ...       ...       ...       ...   
2020-12-31 93426.0   31.480000     12527.0  0.076239   12552.0  0.076239   
           93427.0   77.589996     40249.0  0.135851   36938.0  0.135851   
           93429.0   93.120003    167295.0  0.019711  108042.0  0.019711   
           93434.0    2.930000     21454.0  0.122605   33482.0  0.122605   
           93436.0  705.669983  11962716.0  0.243252  947901.0  0.243252   

                    ret_ma1_lag   ret_ma2  ret_ma2_lag   ret_ma3  ret_ma3_lag  \
DATE       PERMNO                                                               
1993-01-29 10001.0          NaN       NaN          NaN       NaN          NaN   
           10002.0          NaN       NaN          NaN       NaN          NaN   
           10003.0          NaN       NaN          NaN       NaN          NaN   
           10009.0          NaN       NaN          NaN       NaN          NaN   
           10010.0          NaN       NaN          NaN       NaN          NaN   
...                         ...       ...          ...       ...          ...   
2020-12-31 93426.0     0.224362  0.150300     0.083942  0.081375     0.061448   
           93427.0     0.138121  0.136986     0.045183  0.075406    -0.002122   
           93429.0     0.128552  0.074132     0.027520  0.024917     0.003639   
           93434.0     0.144737  0.133671     0.032046  0.062232     0.039804   
           93436.0     0.462736  0.352994     0.183618  0.203496     0.076050   

                    ...   ret_ma6  ret_ma6_lag   ret_ma7  ret_ma7_lag  \
DATE       PERMNO   ...                                                 
1993-01-29 10001.0  ...       NaN          NaN       NaN          NaN   
           10002.0  ...       NaN          NaN       NaN          NaN   
           10003.0  ...       NaN          NaN       NaN          NaN   
           10009.0  ...       NaN          NaN       NaN          NaN   
           10010.0  ...       NaN          NaN       NaN          NaN   
...                 ...       ...          ...       ...          ...   
2020-12-31 93426.0  ...  0.045793     0.039861  0.045058     0.037256   
           93427.0  ...  0.042302     0.015698  0.032863     0.016165   
           93429.0  ...  0.003706    -0.020213 -0.014510    -0.006631   
           93434.0  ...  0.045422     0.036724  0.048993     0.048771   
           93436.0  ...  0.256311     0.264633  0.261579     0.236534   

                     ret_ma8  ret_ma8_lag   ret_ma9  ret_ma9_lag  ret_ma10  \
DATE       PERMNO                                                            
1993-01-29 10001.0       NaN          NaN       NaN          NaN       NaN   
           10002.0       NaN          NaN       NaN          NaN       NaN   
           10003.0       NaN          NaN       NaN          NaN       NaN   
           10009.0       NaN          NaN       NaN          NaN       NaN   
           10010.0       NaN          NaN       NaN          NaN       NaN   
...                      ...          ...       ...          ...       ...   
2020-12-31 93426.0  0.042129     0.051523  0.054269     0.015907  0.021940   
           93427.0  0.031126     0.032908  0.044346     0.028123  0.038896   
           93429.0 -0.003338     0.008386  0.009644    -0.016669 -0.013031   
           93434.0  0.058001     0.033529  0.043426    -0.004095  0.008575   
           93436.0  0.237374     0.268484  0.265681     0.214702  0.217557   

                   

In [15]:
len_df = len(df)
len_df

2471978

#### Strategy Analysis

For the reversal strategy, we calculate the quantile based on past returns.

In order to check the feasibility of reversal strategy on ALL ETFs, 
we first check its performance using backtest.

In [16]:
def strategy_analysis(data, q=5, N=5, crit_etf=slice(None)):
    def quantile(inser, q):
        return pd.qcut(inser, q, labels=range(1, q+1), duplicates='drop')

    crit_ret = f'ret_ma{N}_lag'
    tmp = data.loc[crit_etf].copy()
    tmp = tmp[['PRC', 'VOL', 'RET', 'SHROUT', crit_ret]]
    # drop missing values
    tmp.dropna(subset=[f'ret_ma{N}_lag'], inplace=True)
    assert tmp.isna().sum().sum()==0, 'Missing Values exist!'
    
    tmp['quantile'] = tmp[crit_ret].groupby('DATE').apply(quantile, q)
    ports = tmp.groupby(['quantile','DATE'])['RET'].mean()
    # Using reversal strategy
    long_short = ports.loc[1] - ports.loc[q]
    long_short = long_short.describe()
    long_short['sharpe_annual'] = long_short['mean'] / long_short['std'] * np.sqrt(12)
    long_short['tstat'] = long_short['mean'] / (long_short['std'] / np.sqrt(long_short['count']))
    
    return ports, long_short

def param_tuning(data, q_range, N_range, crit_etf=slice(None)):
    q_ranges, N_ranges, means, sharpes, tstats, significances = [], [], [], [], [], []
    for q in q_range:
        for N in N_range:
            ports, long_short = strategy_analysis(data, q, N, crit_etf)
            q_ranges.append(q)
            N_ranges.append(N)
            means.append(long_short['mean'])
            sharpes.append(long_short['sharpe_annual'])
            tstats.append(long_short['tstat'])
            significances.append(abs(long_short['tstat'])>=1.96)
    result_df = pd.DataFrame({'q': q_ranges, 'N': N_ranges, 'Mean': means,  
                              'Sharpe_Annual': sharpes, 'tstat': tstats, 
                              'significance': significances})
    return result_df

Backtest for Tuning

In [17]:
param_tuning(df, q_range=range(2,6), N_range=range(1,4))

,q,N,Mean,Sharpe_Annual,tstat,significance
0,2,1,0.003487,0.363527,1.920741,False
1,2,2,0.002155,0.206336,1.088574,False
2,2,3,0.000306,0.029382,0.154778,False
3,3,1,0.005337,0.404563,2.137558,True
4,3,2,0.003081,0.214593,1.132133,False
5,3,3,0.000591,0.040947,0.215701,False
6,4,1,0.006962,0.447410,2.363945,True
7,4,2,0.004360,0.258138,1.361867,False
8,4,3,0.001218,0.071905,0.378783,False
9,5,1,0.008643,0.499201,2.637589,True


**Insights:** 

(1) large *N* leads to un-significant results, thus testing on small *N*s

(2) large *q* leads to higher returns, thus testing on larger *q*s

In [18]:
param_tuning(df, q_range=range(5,11), N_range=range(1,3))

,q,N,Mean,Sharpe_Annual,tstat,significance
0,5,1,0.008643,0.499201,2.637589,True
1,5,2,0.005562,0.296939,1.566568,False
2,6,1,0.010087,0.541796,2.862645,True
3,6,2,0.006886,0.341215,1.800157,False
4,7,1,0.011707,0.589320,3.113744,True
5,7,2,0.008210,0.384911,2.030687,True
6,8,1,0.012910,0.621360,3.283030,True
7,8,2,0.009419,0.421283,2.222576,True
8,9,1,0.014300,0.659345,3.483729,True
9,9,2,0.010485,0.451481,2.381890,True


**Insights:** large *q* with small *N* leads to better performance.

Therefore, we produce quantiles using large *q*s and small *N*s, and further rigorously tune hyperparameters during the combination phase. 

#### Strategy Outputs

In this part, we produce quantile labels using reversal strategy, which serve as the criterion for further portfolio selection in the combination phase.

In [19]:
def quantile_labels(data, q=5, N=5, crit_etf=slice(None)):
    def quantile(inser, q):
        return pd.qcut(inser, q, labels=range(1,q+1), duplicates='drop')
    
    crit_ret = f'ret_ma{N}_lag'
    tmp = data.loc[crit_etf].copy()
    tmp = tmp[['PRC', 'VOL', 'RET', 'SHROUT', crit_ret]]
    # drop missing values
    tmp.dropna(subset=[f'ret_ma{N}_lag'], inplace=True)
    assert tmp.isna().sum().sum()==0, 'Missing Values exist!'
        
    tmp[f'quantile_q{q}_ma{N}'] = tmp[crit_ret].groupby('DATE').apply(quantile, q)
    
    return tmp[[f'quantile_q{q}_ma{N}']]

Output Quantiles

aim for a flexible combined strategy, produce quantile labels based on different hyperparameter settings

In [20]:
##### merge is slow!!!
results = pd.DataFrame(index=df.index)
results.reset_index(inplace=True)
assert len(results)==len_df, 'Missing Some Rows!'

q_range, N_range = range(5,11), range(1,3)
for q in q_range:
    for N in N_range:
        tmp_result = quantile_labels(df, q, N, crit_etf=slice(None))
        tmp_result.reset_index(inplace=True)
        results = results.merge(tmp_result, how='outer', on=['DATE', 'PERMNO'])
        print(f'work done: q{q}, N{N}')
results

work done: q5, N1
work done: q5, N2
work done: q6, N1
work done: q6, N2
work done: q7, N1
work done: q7, N2
work done: q8, N1
work done: q8, N2
work done: q9, N1
work done: q9, N2
work done: q10, N1
work done: q10, N2


,DATE,PERMNO,quantile_q5_ma1,quantile_q5_ma2,quantile_q6_ma1,quantile_q6_ma2,quantile_q7_ma1,quantile_q7_ma2,quantile_q8_ma1,quantile_q8_ma2,quantile_q9_ma1,quantile_q9_ma2,quantile_q10_ma1,quantile_q10_ma2
0,1993-01-29,10001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-01-29,10002.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-01-29,10003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-01-29,10009.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-01-29,10010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471973,2020-12-31,93426.0,4,4,5,4,6,5,7,5,7,6,8,7
2471974,2020-12-31,93427.0,3,3,4,3,4,3,5,4,5,4,6,5
2471975,2020-12-31,93429.0,3,2,4,2,4,3,5,3,5,3,6,3
2471976,2020-12-31,93434.0,3,2,4,2,4,3,5,3,6,3,6,4


In [21]:
len(results)==len_df

True

df_reversal stores the result of reversal strategy.

In [23]:
df_reversal = results.copy()

### Payout Yield

The “conservative formula” of VVB combines three ingredients：Low risk, High past returns and High payout ratio.
Since we already include volatility and momentum in this notebook, here, we mainly focus on the third ingredient to avoid redundency.

**Data Cleaning**

In [24]:
data = data.set_index(['PERMNO','DATE'])
df = data.copy()
df.head()

PRC  SHROUT  SHRCD  EXCHCD   SICCD    VOL       RET  \
PERMNO  DATE                                                                 
10001.0 1993-01-29 -14.000  1080.0   11.0     3.0  4920.0   44.0  0.000000   
        1993-02-26  14.250  1080.0   11.0     3.0  4920.0   23.0  0.017857   
        1993-03-31  14.250  1075.0   11.0     3.0  4920.0  137.0  0.011053   
        1993-04-30 -15.250  1075.0   11.0     3.0  4920.0  473.0  0.070175   
        1993-05-28 -15.125  1075.0   11.0     3.0  4920.0  126.0 -0.008197   

                    DIVAMT         MV  
PERMNO  DATE                           
10001.0 1993-01-29     NaN -15120.000  
        1993-02-26     NaN  15390.000  
        1993-03-31  0.1575  15318.750  
        1993-04-30     NaN -16393.750  
        1993-05-28     NaN -16259.375

In [25]:
# Only keep common stocks 
df = df.loc[(df['SHRCD']==10) | (df['SHRCD']==11)]
df.drop('SHRCD', axis=1, inplace=True)
# Fix negative stock price
df['PRC'] = np.abs(df['PRC'])

df.sort_index(inplace=True)
df.dropna(subset=['EXCHCD','PRC','VOL','SHROUT'], inplace=True)
df = df.groupby(['PERMNO','DATE']).head(1)
df.head()

PRC  SHROUT  EXCHCD   SICCD    VOL       RET  DIVAMT  \
PERMNO  DATE                                                                  
10001.0 1993-01-29  14.000  1080.0     3.0  4920.0   44.0  0.000000     NaN   
        1993-02-26  14.250  1080.0     3.0  4920.0   23.0  0.017857     NaN   
        1993-03-31  14.250  1075.0     3.0  4920.0  137.0  0.011053  0.1575   
        1993-04-30  15.250  1075.0     3.0  4920.0  473.0  0.070175     NaN   
        1993-05-28  15.125  1075.0     3.0  4920.0  126.0 -0.008197     NaN   

                           MV  
PERMNO  DATE                   
10001.0 1993-01-29 -15120.000  
        1993-02-26  15390.000  
        1993-03-31  15318.750  
        1993-04-30 -16393.750  
        1993-05-28 -16259.375

In [26]:
df_payout = df.copy()

Define lags of variables

In [27]:
df['lag_PRC']    = df['PRC'].groupby('PERMNO').shift(1)
df['lag_SHROUT'] = df['SHROUT'].groupby('PERMNO').shift(1)
df['lag_MV']     = df['MV'].groupby('PERMNO').shift(1)

**Compute the lagged payout yield in different senarios**

- Dividends (dividend per share x number of shares)
- Net repurchases (-1 x change in shares outstanding x price per share)

In [28]:
df['DIVAMT'] = df['DIVAMT'].fillna(0)
df['PAYOUT'] = df['SHROUT']*df['DIVAMT'] + np.maximum(df['lag_SHROUT']-df['SHROUT'], 0)*df['PRC']

In [29]:
def quintiles(inser, i=5):
    outser = pd.qcut(inser, q=i, labels=range(1,i+1))
    return outser

In [30]:
# Produce quantiles of different combinition of lags and quantile
# This cell takes about 5 minutes to run
for i in [3,6,9,12]:
    for j in [3,5,10]:
        df[f'lag_PAYOUT_{i}m'] = df['PAYOUT'].groupby('PERMNO').rolling(i).sum().shift(1).reset_index(level=0, drop=True)
        df[f'lag_PAYOUTYIELD_{i}m'] = df[f'lag_PAYOUT_{i}m']/df['lag_MV']
        # Since payout yield are likely to be 0, we add some randomness intentionally to avoid duplicate bin edges.
        df[f'lag_PAYOUTYIELD_{i}m'] = df[f'lag_PAYOUTYIELD_{i}m'] + df['lag_PRC']/10000000000
        # Compute quintile rank
        df[f'PAYOUT_QUINTILE_{i}m_{j}'] =  df.dropna(subset=[f'lag_PAYOUTYIELD_{i}m','RET'])[f'lag_PAYOUTYIELD_{i}m'].groupby('DATE').apply(quintiles, j)
    print(f'Lag {i} months done.')

Lag 3 months done.
Lag 6 months done.
Lag 9 months done.
Lag 12 months done.


In [31]:
df.tail()

PRC    SHROUT  EXCHCD   SICCD         VOL  \
PERMNO  DATE                                                           
93436.0 2020-08-31  498.320007  931809.0     3.0  9999.0   4051970.0   
        2020-09-30  429.010010  948000.0     3.0  9999.0  17331954.0   
        2020-10-30  388.040009  947901.0     3.0  9999.0   8330610.0   
        2020-11-30  567.599976  947901.0     3.0  9999.0   7811501.0   
        2020-12-31  705.669983  947901.0     3.0  9999.0  11962716.0   

                         RET  DIVAMT            MV      lag_PRC  lag_SHROUT  \
PERMNO  DATE                                                                  
93436.0 2020-08-31  0.741452     0.0  4.643391e+08  1430.760010    186362.0   
        2020-09-30 -0.139087     0.0  4.067015e+08   498.320007    931809.0   
        2020-10-30 -0.095499     0.0  3.678235e+08   429.010010    948000.0   
        2020-11-30  0.462736     0.0  5.380286e+08   388.040009    947901.0   
        2020-12-31  0.243252     0.0  6.689053e+08   567.599976    947901.0   

                    ...  lag_PAYOUT_9m  lag_PAYOUTYIELD_9m  \
PERMNO  DATE        ...                                      
93436.0 2020-08-31  ...       0.000000        1.430760e-07   
        2020-09-30  ...       0.000000        4.983200e-08   
        2020-10-30  ...       0.000000        4.290100e-08   
        2020-11-30  ...   38415.960846        1.044801e-04   
        2020-12-31  ...   38415.960846        7.145810e-05   

                    PAYOUT_QUINTILE_9m_3  PAYOUT_QUINTILE_9m_5  \
PERMNO  DATE                                                     
93436.0 2020-08-31                     1                     2   
        2020-09-30                     2                     2   
        2020-10-30                     2                     2   
        2020-11-30                     2                     2   
        2020-12-31                     2                     2   

                   PAYOUT_QUINTILE_9m_10 lag_PAYOUT_12m lag_PAYOUTYIELD_12m  \
PERMNO  DATE                                                                  
93436.0 2020-08-31                     4       0.000000        1.430760e-07   
        2020-09-30                     4       0.000000        4.983200e-08   
        2020-10-30                     4       0.000000        4.290100e-08   
        2020-11-30                     4   38415.960846        1.044801e-04   
        2020-12-31                     4   38415.960846        7.145810e-05   

                    PAYOUT_QUINTILE_12m_3  PAYOUT_QUINTILE_12m_5  \
PERMNO  DATE                                                       
93436.0 2020-08-31                      1                      2   
        2020-09-30                      1                      2   
        2020-10-30                      1                      2   
        2020-11-30                      1                      2   
        2020-12-31                      1                      2   

                   PAYOUT_QUINTILE_12m_10  
PERMNO  DATE                               
93436.0 2020-08-31                      3  
        2020-09-30                      3  
        2020-10-30                      3  
        2020-11-30                      4  
        2020-12-31                      4  

[5 rows x 32 columns]

#### Strategy Analysis

Tune hyperparameters (quantile, lag) to achieve the highest return:

In [32]:
lags, qs, longs, shorts = [], [], [], []
for i in [3,6,9,12]:
    for j in [3,5,10]:
        payout_ports = df.groupby([f'PAYOUT_QUINTILE_{i}m_{j}','DATE'])['RET'].mean()
        payout_stats = payout_ports.groupby([f'PAYOUT_QUINTILE_{i}m_{j}']).describe()
        payout_stats['sharpe'] = payout_stats['mean'] / payout_stats['std'] * np.sqrt(12)
        lags.append(i)
        qs.append(j)
        shorts.append(payout_stats.iloc[0,1])
        longs.append(payout_stats.iloc[-1,1])

In [33]:
tuning_df = pd.DataFrame({'lags':lags, 'q':qs, 'long':longs, 'short':shorts})
tuning_df['long-short'] = tuning_df['long'] - tuning_df['short']

In [34]:
tuning_df.sort_values(by='long-short', ascending=False)[:3]

,lags,q,long,short,long-short
9,12,3,0.012480,0.011676,0.000805
6,9,3,0.012264,0.011759,0.000505
3,6,3,0.012208,0.012316,-0.000108


Based on the long-short result of different combination of hyperparameters, we can see that the payout yield strategy does not perform well. 
- The lowest quantile's return is always  positive
- The difference between the highest and lowest quantile is more likely to be negative. 

Here, we choose lag=12 and q=3.

We can expect that, when we combine all strategies together, payout yield strategy will be assigned with a low weight.

#### Strategy Outputs

In [35]:
df.dropna(subset=['PAYOUT_QUINTILE_12m_3'], inplace=True)
data['Payout_QUINTILE'] = df['PAYOUT_QUINTILE_12m_3'].astype(int)
data['Payout_QUINTILE'] = data['Payout_QUINTILE'].fillna(0)

The series returned in this strategy is ```data['PAYOUT_QUINTILE']``` where 0 represents no action, 3 represents long and 1 represents short. 

In [36]:
data['Payout_QUINTILE'].value_counts()

0.0    1083913
1.0     491729
3.0     491319
2.0     490910
Name: Payout_QUINTILE, dtype: int64

### Volatility

In [37]:
df = df_payout.copy()

In [38]:
# compute 1, 2 and 3 years of lags
# Produce quantiles of different combinition of lags and quantile
for i in [12,24,36]:
    for j in [3,5,10]:
        df[f'lag_SD_{i}m'] = df['RET'].groupby('PERMNO').rolling(i).std().shift(1).reset_index(level=0, drop=True)
        # Compute quintile rank
        df[f'Volatility_QUINTILE_{i}m_{j}'] =  df[f'lag_SD_{i}m'].groupby('DATE').apply(quintiles, j)
    print(f'Lag {i} months done.')

Lag 12 months done.
Lag 24 months done.
Lag 36 months done.


#### Strategy Analysis

In [39]:
# Tune hyperparameters (quantile, lag) to achieve the highest return
lags, qs, longs, shorts = [], [], [], []
for i in [12,24,36]:
    for j in [3,5,10]:
        vola_ports = df.groupby([f'Volatility_QUINTILE_{i}m_{j}','DATE'])['RET'].mean()
        vola_stats = vola_ports.groupby([f'Volatility_QUINTILE_{i}m_{j}']).describe()
        vola_stats['sharpe'] = vola_stats['mean'] / vola_stats['std'] * np.sqrt(12)
        lags.append(i)
        qs.append(j)
        shorts.append(vola_stats.iloc[-1,1])
        longs.append(vola_stats.iloc[0,1])

In [40]:
tuning_df = pd.DataFrame({'lags':lags, 'q':qs, 'long':longs, 'short':shorts})
tuning_df['long-short'] = tuning_df['long'] - tuning_df['short']

In [41]:
tuning_df.sort_values(by='long-short', ascending=False)[:3]

,lags,q,long,short,long-short
2,12,10,0.010581,0.010351,0.000230
5,24,10,0.010988,0.012058,-0.001070
8,36,10,0.010366,0.011744,-0.001378


According to the above table, 12 months lags with 10 quatiles is the combination with highest returns

#### Strategy Outputs

In [42]:
df.dropna(subset=['Volatility_QUINTILE_12m_10'], inplace=True)
data['Volatility_QUINTILE'] = df['Volatility_QUINTILE_12m_10'].astype(int)
data['Volatility_QUINTILE'] = data['Volatility_QUINTILE'].fillna(0)

In [43]:
# 0 represents no action, 5 represents long and 1 represents short.
data['Volatility_QUINTILE'].value_counts()

0.0     1068009
1.0      149135
10.0     149103
5.0      148998
8.0      148977
2.0      148974
4.0      148967
3.0      148937
9.0      148934
7.0      148927
6.0      148910
Name: Volatility_QUINTILE, dtype: int64

### Momentum

In [44]:
def weighted_average(df,data_col,weight_col,by_col):
    df['_data_times_weight'] = df[data_col]*df[weight_col]
    df['_weight_where_notnull'] = df[weight_col]*pd.notnull(df[data_col])
    g = df.groupby(by_col)
    result = g['_data_times_weight'].sum() / g['_weight_where_notnull'].sum()
    del df['_data_times_weight'], df['_weight_where_notnull']
    return result

In [45]:
rf = pd.read_csv('./Data/risk-free_return.csv', parse_dates=[0], index_col=[0])
rf.head()

,RF
DATE,
1926-07-31,0.0022
1926-08-31,0.0025
1926-09-30,0.0023
1926-10-31,0.0032
1926-11-30,0.0031


In [46]:
stocks = df_payout.copy()
stocks['lagMV'] = stocks.groupby('PERMNO')['MV'].shift()

#### Strategy Analysis

In [47]:
def momentum_tuning(df, minlag, maxlag, weighttype, pastrettype, numbins):
    if pastrettype=='comp':
        df['CUMGROSSRET'] = (1+df['RET']).groupby('PERMNO').cumprod()
        df['MOM'] = df['CUMGROSSRET'].groupby('PERMNO').shift(minlag) / df['CUMGROSSRET'].groupby('PERMNO').shift(maxlag) - 1 
    if pastrettype=='mean':
        df['ROLLMEAN'] = df['RET'].groupby('PERMNO').rolling(maxlag-minlag).mean().reset_index(level=0, drop=True)
        df['MOM'] = df['ROLLMEAN'].groupby('PERMNO').shift(minlag)   
    if pastrettype=='sharpe':
        df['ROLLMEAN'] = df['RET'].groupby('PERMNO').rolling(maxlag-minlag).mean().reset_index(level=0, drop=True)
        df['ROLLSD'] = df['RET'].groupby('PERMNO').rolling(maxlag-minlag).std().reset_index(level=0, drop=True)
        df['MOM'] = df['ROLLMEAN'].groupby('PERMNO').shift(minlag) / df['ROLLSD'].groupby('PERMNO').shift(minlag)
    df.dropna(subset=['RET','MOM','MV'], inplace=True)
    df['XRET'] = df['RET'] - rf['RF']/100
    df = df.reorder_levels(['DATE','PERMNO'])
    df.sort_index(inplace=True)
    df['QUANTILE'] = df['MOM'].groupby('DATE').apply(quintiles, numbins)
    if weighttype=='EW':
        ports = df.groupby(['QUANTILE','DATE'])['XRET'].mean()
    if weighttype=='VW':
        ports = weighted_average(df, 'XRET', 'lagMV', ['QUANTILE','DATE'])

    mom_ports = df.groupby(['QUANTILE','DATE'])['RET'].mean()
    mom_stats = mom_ports.groupby('QUANTILE').describe()
    mom_stats['sharpe'] = mom_stats['mean'] / mom_stats['std'] * np.sqrt(12)
    short = mom_stats.iloc[0,1]
    long = mom_stats.iloc[-1,1]
    #display(mom_stats)
    
    return minlag, maxlag, weighttype, pastrettype, numbins, short, long

In [48]:
minlags, maxlags, weighttypes, pastrettypes, numbinses, shorts, longs = [],[],[],[],[],[],[]
for minlag in [1,2,3]:
    for maxlag in [10,13,20]:
        for weighttype in ['VW','EW']:
            for pastrettype in ['mean','comp','sharpe']:
                for numbins in [5, 10]:
                    df = stocks.copy()
                    #print(minlag,maxlag,weighttype,pastrettype,numbins)
                    l = momentum_tuning(df,minlag,maxlag,weighttype,pastrettype,numbins)
                    minlags.append(l[0])
                    maxlags.append(l[1])
                    weighttypes.append(l[2])
                    pastrettypes.append(l[3]) 
                    numbinses.append(l[4])
                    shorts.append(l[5])
                    longs.append(l[6])
        print(f'{minlag},{maxlag} done')

1,10 done
1,13 done
1,20 done
2,10 done
2,13 done
2,20 done
3,10 done
3,13 done
3,20 done


In [49]:
tuning_df = pd.DataFrame({'minlags':minlags, 'maxlags':maxlags, 'weighttypes':weighttypes, 
                          'pastrettypes':pastrettypes, 'numbinses':numbinses, 'shorts':shorts, 'longs':longs})
tuning_df['long-short'] = tuning_df['longs'] - tuning_df['shorts']

In [50]:
tuning_df.sort_values(by='long-short', ascending=False).head(1)

,minlags,maxlags,weighttypes,pastrettypes,numbinses,shorts,longs,long-short
88,3,13,VW,sharpe,5,0.009624,0.014982,0.005359


According to the above table, minlag=3,maxlag=13,weighttype='VW',pastrettype='sharpe',numbins=5 is the combination with highest returns

In [51]:
minlag=3
maxlag=13
weighttype='VW'
pastrettype='sharpe'
numbins=5

if pastrettype=='sharpe':
    stocks['ROLLMEAN'] = stocks['RET'].groupby('PERMNO').rolling(maxlag-minlag).mean().reset_index(level=0, drop=True)
    stocks['ROLLSD'] = stocks['RET'].groupby('PERMNO').rolling(maxlag-minlag).std().reset_index(level=0, drop=True)
    stocks['MOM'] = stocks['ROLLMEAN'].groupby('PERMNO').shift(minlag) / stocks['ROLLSD'].groupby('PERMNO').shift(minlag)
stocks.dropna(subset=['RET','MOM','MV'], inplace=True)
stocks['XRET'] = stocks['RET'] - rf['RF']/100
stocks = stocks.reorder_levels(['DATE','PERMNO'])
stocks.sort_index(inplace=True)
stocks['QUANTILE'] = stocks['MOM'].groupby('DATE').apply(quintiles, numbins)
if weighttype=='VW':
    ports = weighted_average(stocks, 'XRET', 'lagMV', ['QUANTILE','DATE'])

mom_ports = stocks.groupby(['QUANTILE','DATE'])['RET'].mean()
mom_stats = mom_ports.groupby('QUANTILE').describe()
mom_stats['sharpe'] = mom_stats['mean'] / mom_stats['std'] * np.sqrt(12)
short = mom_stats.iloc[0,1]
long = mom_stats.iloc[-1,1]
display(mom_stats)

,count,mean,std,min,25%,50%,75%,max,sharpe
QUANTILE,,,,,,,,,
1,324.0,0.009624,0.079159,-0.243735,-0.033915,0.011436,0.048104,0.488295,0.421151
2,324.0,0.010640,0.063577,-0.241448,-0.024542,0.011897,0.043747,0.307921,0.579743
3,324.0,0.011134,0.057700,-0.239963,-0.020078,0.014960,0.044023,0.216043,0.668437
4,324.0,0.013295,0.055763,-0.220837,-0.012514,0.016151,0.047476,0.187682,0.825905
5,324.0,0.014982,0.054329,-0.191137,-0.009231,0.017540,0.045902,0.213700,0.955307


#### Strategy Outputs

In [52]:
stocks = stocks.reorder_levels(['PERMNO','DATE'])

In [53]:
stocks.dropna(subset=['QUANTILE'], inplace=True)
data['Momentum_QUINTILE'] = stocks['QUANTILE'].astype(int)
data['Momentum_QUINTILE'] = data['Momentum_QUINTILE'].fillna(0)

In [54]:
# 0 represents no action, 5 represents long and 1 represents short.
data['Momentum_QUINTILE'].value_counts()

0.0    1082569
1.0     295191
5.0     295127
3.0     294996
4.0     294994
2.0     294994
Name: Momentum_QUINTILE, dtype: int64

## Final Results
Merge all strategy results to data

In [55]:
df_reversal = df_reversal.set_index(['PERMNO','DATE'])

In [56]:
output = data.join(df_reversal, how='left')

In [57]:
output.columns

Index(['PRC', 'SHROUT', 'SHRCD', 'EXCHCD', 'SICCD', 'VOL', 'RET', 'DIVAMT',
       'MV', 'Payout_QUINTILE', 'Volatility_QUINTILE', 'Momentum_QUINTILE',
       'quantile_q5_ma1', 'quantile_q5_ma2', 'quantile_q6_ma1',
       'quantile_q6_ma2', 'quantile_q7_ma1', 'quantile_q7_ma2',
       'quantile_q8_ma1', 'quantile_q8_ma2', 'quantile_q9_ma1',
       'quantile_q9_ma2', 'quantile_q10_ma1', 'quantile_q10_ma2'],
      dtype='object')

In [58]:
output.to_csv('./Data/output.csv')